### Required Libraries

In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,roc_curve
from functions import decile_analysis
print('********* Required Libraries Imported *********')

********* Required Libraries Imported *********


### Load Dataset

In [2]:
#load model developing data
model_data = pd.read_parquet('Leobrix_Release_BBILAP_Combined_Data.parquet.gzip')
print('********* Data loaded ********* ')
print(model_data.shape)
# Make sure no duplicates in the data
model_data =  model_data.drop_duplicates(subset=['unique_reference_number'])
model_data =  model_data.drop_duplicates(subset=['latest_feature_id'])
print(model_data.shape)

********* Data loaded ********* 
(478328, 2831)
(478328, 2831)


### Drop Null target records

In [3]:
model_data['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
NaN    354961
0.0    109935
1.0     13432
Name: count, dtype: int64

In [4]:
# Drop Null values of Target 
df = model_data.dropna(subset=['60_in_15mob'])
# take one copy for model development data
data = df.copy()
#some bank data have inf very rate but by safe replace 
data.replace([np.inf, -np.inf], np.nan, inplace=True)

### Split the Time interval

In [5]:
model_df1 = data[(data['month_year']>='2021-01') & (data['month_year']<='2023-03')]
oot_df1= data[(data['month_year']>='2023-04') & (data['month_year']<='2023-06')]

In [6]:
model_df1.shape, oot_df1.shape

((101984, 2831), (17343, 2831))

In [7]:
features_list = [

#Bureau Intutive
    
'num_PERSONAL LOAN_enq_last_24M',
'num_CONSUMER LOAN_enq_last_9M',
'num_of_loans_le_1lac_non_credit',
'num_CREDIT CARD_enq_last_3M',
'Utilization_all',
'Total_outstanding_Balance_CREDIT CARD_opened_last_6M',
'Age_of_oldest_open_account',
'highest_sanction_amount_non_credit',
'Total_high_credit',
'num_PL_LE_1_Lac_opened_in_last_3M',

# banking intutive
    
'cash_withdrawal_sum',
'bounced_transactions_count',
'num_credit_gt_salary',
'upi_trans_count',
'ratio_total_debit_to_total_credit_M3',
'ratio_6th_day_balance_to_2th_day_balance_M1',

                 
# Reconcilated Features

'percentage_ever_delinquent', 
'months_since_most_recent_delinquency_all',
'percentage_utilization_gt_75',
'No of enq in past 90days',
'Num_Enq_non_BANK_last_12M',
'Num_of_installment_loan_opened',


'dm216s',
'dm206s',
'g406s',
'balmag01',
    
    
'month_year',
'60_in_15mob']

In [8]:
len(features_list)

28

In [9]:
# Selected Features for model build purpose
model_df = model_df1[features_list]

In [10]:
oot_df = oot_df1[features_list]

In [11]:
model_df.shape, oot_df.shape

((101984, 28), (17343, 28))

In [12]:
model_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    90856
1.0    11128
Name: count, dtype: int64

In [13]:
oot_df['60_in_15mob'].value_counts(dropna=False)

60_in_15mob
0.0    15505
1.0     1838
Name: count, dtype: int64

In [14]:
# Separate features (X) and target (y) for Model and OOT datasets
X_model = model_df.drop(columns=['month_year', '60_in_15mob']) # Adjust 'target' to your actual target column name
y_model = model_df['60_in_15mob']

In [15]:
X_oot = oot_df.drop(columns=['month_year', '60_in_15mob'])
y_oot = oot_df['60_in_15mob']

In [16]:
#Handling Null Values
for i in X_model.columns:
    X_model[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3030057/2233446063.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_model[i].fillna(-9999,inplace=True)


In [17]:
for i in X_oot.columns:
    X_oot[i].fillna(-9999,inplace=True)

/tmp/ipykernel_3030057/1979277635.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_oot[i].fillna(-9999,inplace=True)


In [18]:
X_oot.shape,y_oot.shape

((17343, 26), (17343,))

### Split Model dataset into Train and Test

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X_model, y_model, test_size=0.40,random_state=42)

In [20]:
X_train.shape, X_test.shape

((61190, 26), (40794, 26))

In [21]:
#Split Training data into 80% train and 20% eval
X_train_new, X_eval, y_train_new, y_eval = train_test_split(
    X_train, y_train, test_size=0.2, random_state =42
)
print(X_train_new.shape, X_eval.shape, X_test.shape)

(48952, 26) (12238, 26) (40794, 26)


In [22]:
# 2. **Prepare DMatrix for XGBoost**
dtrain = xgb.DMatrix(X_train_new, label=y_train_new)
deval = xgb.DMatrix(X_eval, label=y_eval)
dtest = xgb.DMatrix(X_test, label=y_test)
doot = xgb.DMatrix(X_oot, label=y_oot)


In [23]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'learning_rate': 0.01,
     'max_depth': 2,
    'subsample': 0.6,
    'colsample_bytree': 0.6,

    'lambda': 5.3,
    'alpha': 4.6,
    'gamma': 4.8,
   
    'scale_pos_weight': 3.0,
    'random_state': 24
    
}



evals = [(dtrain, 'train'), (deval, 'eval')]
xgb_model = xgb.train(
    params,
    dtrain,
    num_boost_round= 700,
    early_stopping_rounds= 30,
    evals=evals,
    verbose_eval=10
)

X_train_pred = X_train_new.copy()
X_eval_pred = X_eval.copy()
X_test_pred = X_test.copy()
X_oot_pred = X_oot.copy()


X_train_pred['y_actual'] = y_train_new
X_train_pred['y_pred'] = xgb_model.predict(dtrain)
X_train_pred['LID'] = model_df1.loc[X_train_new.index,'latest_feature_id'].values

X_eval_pred['y_actual'] = y_eval
X_eval_pred['y_pred'] = xgb_model.predict(deval)
X_eval_pred['LID'] = model_df1.loc[X_eval.index,'latest_feature_id'].values

X_test_pred['y_actual'] = y_test
X_test_pred['y_pred'] = xgb_model.predict(dtest)
X_test_pred['LID'] = model_df1.loc[X_test.index,'latest_feature_id'].values

X_oot_pred['y_actual'] = y_oot
X_oot_pred['y_pred'] = xgb_model.predict(doot)
X_oot_pred['LID'] = oot_df1.loc[X_oot.index,'latest_feature_id'].values

[0]	train-auc:0.62209	eval-auc:0.61738
[10]	train-auc:0.65423	eval-auc:0.64392
[20]	train-auc:0.65364	eval-auc:0.64387
[30]	train-auc:0.65758	eval-auc:0.64822
[40]	train-auc:0.65870	eval-auc:0.64886
[50]	train-auc:0.65972	eval-auc:0.64873
[60]	train-auc:0.66155	eval-auc:0.65039
[70]	train-auc:0.66172	eval-auc:0.65073
[80]	train-auc:0.66227	eval-auc:0.65158
[90]	train-auc:0.66302	eval-auc:0.65204
[100]	train-auc:0.66416	eval-auc:0.65360
[110]	train-auc:0.66453	eval-auc:0.65454
[120]	train-auc:0.66538	eval-auc:0.65539
[130]	train-auc:0.66581	eval-auc:0.65589
[140]	train-auc:0.66638	eval-auc:0.65629
[150]	train-auc:0.66721	eval-auc:0.65670
[160]	train-auc:0.66788	eval-auc:0.65720
[170]	train-auc:0.66857	eval-auc:0.65780
[180]	train-auc:0.66942	eval-auc:0.65862
[190]	train-auc:0.67000	eval-auc:0.65926
[200]	train-auc:0.67071	eval-auc:0.65983
[210]	train-auc:0.67146	eval-auc:0.66047
[220]	train-auc:0.67208	eval-auc:0.66077
[230]	train-auc:0.67277	eval-auc:0.66137
[240]	train-auc:0.67337	eva

In [24]:
# 6. **Run Decile Analysis**
train_summary, train_auc, train_gini, train_ks = decile_analysis(X_train_pred, 'y_pred', 'y_actual')
eval_summary, eval_auc, eval_gini, eval_ks = decile_analysis(X_eval_pred, 'y_pred', 'y_actual')

test_summary, test_auc, test_gini, test_ks = decile_analysis(X_test_pred, 'y_pred', 'y_actual')
oot_summary, oot_auc, oot_gini, oot_ks = decile_analysis(X_oot_pred, 'y_pred', 'y_actual')

# 7. **Print Results**
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.2f}, KS: {train_ks:.2f}")
print(f"Eval AUC: {eval_auc:.2f}, Gini: {eval_gini:.2f}, KS: {eval_ks:.2f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.2f}, KS: {test_ks:.2f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.2f}, KS: {oot_ks:.2f}")


Train AUC: 0.69, Gini: 0.38, KS: 0.28
Eval AUC: 0.68, Gini: 0.35, KS: 0.26
Test AUC: 0.67, Gini: 0.34, KS: 0.26
OOT AUC: 0.69, Gini: 0.38, KS: 0.28


In [25]:
print(f"Train AUC: {train_auc:.2f}, Gini: {train_gini:.4f}, KS: {train_ks:.4f}")
print(f"Test AUC: {test_auc:.2f}, Gini: {test_gini:.4f}, KS: {test_ks:.4f}")
print(f"OOT AUC: {oot_auc:.2f}, Gini: {oot_gini:.4f}, KS: {oot_ks:.4f}")

Train AUC: 0.69, Gini: 0.3815, KS: 0.2813
Test AUC: 0.67, Gini: 0.3441, KS: 0.2586
OOT AUC: 0.69, Gini: 0.3804, KS: 0.2771


In [26]:
feature_importance = xgb_model.get_score(importance_type='weight')
feature_importance_df = pd.DataFrame(
    list(feature_importance.items()),
    columns =['Feature','Importance']
).sort_values(by='Importance',ascending=False)
print(feature_importance_df)

                                              Feature  Importance
24                                              g406s       234.0
9                   num_PL_LE_1_Lac_opened_in_last_3M       171.0
19                           No of enq in past 90days       151.0
6                          Age_of_oldest_open_account       140.0
4                                     Utilization_all       139.0
20                          Num_Enq_non_BANK_last_12M       132.0
10                                cash_withdrawal_sum       108.0
16                         percentage_ever_delinquent       108.0
1                       num_CONSUMER LOAN_enq_last_9M       100.0
5   Total_outstanding_Balance_CREDIT CARD_opened_l...        93.0
11                         bounced_transactions_count        87.0
18                       percentage_utilization_gt_75        66.0
3                         num_CREDIT CARD_enq_last_3M        61.0
23                                             dm206s        60.0
8         

In [28]:
train_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.277165         0.491280        0.775512        0.404715   4896   
9          0.173442         0.363485        0.404711        0.329603   4895   
8          0.134014         0.304991        0.329601        0.283195   4895   
7          0.116650         0.265318        0.283179        0.249227   4895   
6          0.087845         0.235564        0.249203        0.222757   4895   
5          0.078447         0.211710        0.222750        0.201291   4895   
4          0.074770         0.191907        0.201286        0.183072   4895   
3          0.057814         0.175075        0.183060        0.166923   4895   
2          0.044535         0.158712        0.166910        0.150345   4895   
1          0.034314         0.137235        0.150339        0.103989   4896   

        label_good  label_bad  
Decile                         
10            3539       1357  
9             4046        849  
8             4239        656  
7             4324        571  
6             4465        430  
5             4511        384  
4             4529        366  
3             4612        283  
2             4677        218  
1             4728        168

In [29]:
test_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.261029         0.489981        0.766348        0.403648   4080   
9          0.164256         0.363682        0.403639        0.330057   4079   
8          0.150037         0.305594        0.330034        0.283653   4079   
7          0.115196         0.265890        0.283635        0.249642   4080   
6          0.092425         0.235552        0.249612        0.222679   4079   
5          0.085805         0.211529        0.222678        0.201360   4079   
4          0.082843         0.191917        0.201347        0.183286   4080   
3          0.066928         0.175419        0.183284        0.167190   4079   
2          0.050012         0.159198        0.167187        0.151081   4079   
1          0.037745         0.137819        0.151070        0.098053   4080   

        label_good  label_bad  
Decile                         
10            3015       1065  
9             3409        670  
8             3467        612  
7             3610        470  
6             3702        377  
5             3729        350  
4             3742        338  
3             3806        273  
2             3875        204  
1             3926        154

In [30]:
oot_summary

mean_actual  mean_prediction  max_prediction  min_prediction  count  \
Decile                                                                        
10         0.263401         0.494539        0.751209        0.405890   1735   
9          0.166667         0.364450        0.405765        0.330712   1734   
8          0.140138         0.305918        0.330694        0.282863   1734   
7          0.112457         0.264108        0.282852        0.247792   1734   
6          0.099769         0.233726        0.247749        0.220833   1734   
5          0.089914         0.209515        0.220804        0.199229   1735   
4          0.059977         0.189958        0.199228        0.181284   1734   
3          0.054210         0.173555        0.181265        0.165402   1734   
2          0.041522         0.157644        0.165395        0.149973   1734   
1          0.031700         0.136642        0.149954        0.104599   1735   

        label_good  label_bad  
Decile                         
10            1278        457  
9             1445        289  
8             1491        243  
7             1539        195  
6             1561        173  
5             1579        156  
4             1630        104  
3             1640         94  
2             1662         72  
1             1680         55

In [31]:
#Save the Model
xgb_model.save_model('LeoPayu_BBI_60_in_15mob.json')
print('Model Saved')

Model Saved


In [32]:
#Save the Pred
Train = X_train_pred[['LID','y_actual','y_pred']]
Eval = X_eval_pred[['LID','y_actual','y_pred']]
Test = X_test_pred[['LID','y_actual','y_pred']]
Oot = X_oot_pred[['LID','y_actual','y_pred']]
Train.to_csv('LeoPayu_BBI_Train_Pred.csv',index=False)
Eval.to_csv('LeoPayu_BBI_Eval_Pred.csv',index=False)
Test.to_csv('LeoPayu_BBI_Test_Pred.csv',index=False)
Oot.to_csv('LeoPayu_BBI_Oot_Pred.csv',index=False)
